## The first big project - Professionally You!

### And, Tool use.

### But first: introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and sign up for a free account, and create your API key.

Add to your `.env` file:
```
PUSHOVER_USER=
PUSHOVER_TOKEN=
```
And install the app on your phone.

In [1]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from PyPDF2 import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [4]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [5]:
push("HEY!!")

Push: HEY!!


A function to record if a user wants to be in touch with us

In [6]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

A function to record if a user asks a question that it doesn't know how to answer.

In [7]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

At the end of the day, tool use is just "JSON" and "IF" statements.      
     
Here is some JSON, it records the user details and it refers to the function "record_user_details".      
There is "name" and "description"    
and there are parameters "email" (required), "name" and "notes"     
       
It is way of describing what this function does so that the LLM can decide in its response whether or not it wants to call this tool

In [8]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

This JSON for the function "record_unknown_question"

In [9]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

Final step for this is that we're going to put both of these blobs of JSON into a list of tools.

In [10]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [11]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

In [12]:
print(tools)

[{'type': 'function', 'function': {'name': 'record_user_details', 'description': 'Use this tool to record that a user is interested in being in touch and provided an email address', 'parameters': {'type': 'object', 'properties': {'email': {'type': 'string', 'description': 'The email address of this user'}, 'name': {'type': 'string', 'description': "The user's name, if they provided it"}, 'notes': {'type': 'string', 'description': "Any additional information about the conversation that's worth recording to give context"}}, 'required': ['email'], 'additionalProperties': False}}}, {'type': 'function', 'function': {'name': 'record_unknown_question', 'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer", 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string', 'description': "The question that couldn't be answered"}}, 'required': ['question'], 'additionalProperties': False}}}]


****arguments** - are used in function definition to collect additional keyword arguments as a dictionary      
result = record_user_details(**arguments)       
      
Also, note the "If" statements (Tool is just JSON and IF statemsnts)

In [13]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

globals which gives you a dictionary which you can use to look up any function which is in the global scope.      
     
**globals()** function is used to return the global symbol table – a dictionary representing all the global variables in the current module or script

In [15]:
# globals()

In [19]:
#locals()

In [16]:
globals()["record_unknown_question"]("this is a really hard question")

Push: Recording this is a really hard question asked that I couldn't answer


{'recorded': 'ok'}

So replace the following if statement code with "global"         

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)       

tool = globals().get(tool_name)

In [17]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

We now read the LinkedIn profile And then we bring in the summary text.

In [20]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ed Donner"

In [21]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

Note the part "if you don't know the answer to any question, use your tool to record the question"      
"And if the user is engaging in discussion, try to steer them towards getting in touch via email."      
actually this is not required, as the JSON describes this already but gives kind of context and describes

In [22]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [23]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


"Hi There"       
"What is your job?"        
"Do you have a patent?"      
"Who is your favorite musician?" - should send a message to the phone "Recording who is your favorite musician? asked that I couldn't answer"         
"I'm at csowm5je@yahoo.com" - should send a message to the phone "Recording interest from Name not provided with email csowm5je@yahoo.com and notes not provided"      

### Deployment 
Create an app.py with these code     
It's got the same information in here, but organized into nice little functions at the top of the functions that cover the tools that we call.       
And then there is a "class Me" as in me or or you. And that's something which has the handle tool call that we know so well that's right here.     
      
At the bottom,  we have the Gradio code.       
if __name__ == "__main__":         
    me = Me()      
    gr.ChatInterface(me.chat, type="messages").launch()       
    

#### to run locally

Go to the terminal and      
$> uv run app.py          

It should be running at 127.0.0.1:7861

## And now for deployment

This code is in `app.py`

We will deploy to HuggingFace Spaces:

1. Visit https://huggingface.co and set up an account  
2. From the 1_foundations folder, enter: `gradio deploy`  
3. Follow the instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.

And you're deployed!

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces
  


### Terminal

$> gradio deploy       

Creating new Spaces Repo in       
'D:\Agentic_AI-Engineering\1_foundations'. Collecting metadata,       
press Enter to accept default value.       
Enter Spaces app title [1_foundations]: career_conversation       
Enter Gradio app file [app.py]:        
Enter Spaces hardware (cpu-basic, cpu-upgrade, cpu-xl, zero-a10g, t4-small, t4-medium, l4x1, l4x4, l40sx1, l40sx4, l40sx8, a10g-small, a10g-large, a10g-largex2, a10g-largex4, a100-large, h100, h100x8, v5e-1x1, v5e-2x2, v5e-2x4) [cpu-basic]: cpu-basic       
Any Spaces secrets (y/n) [n]: y      
Enter secret name (leave blank to end): OPENAI_API_KEY      
Enter secret value for OPENAI_API_KEY: sk-proj-      
Enter secret name (leave blank to end): PUSHOVER_USER       
Enter secret value for PUSHOVER_USER: ui      
Enter secret name (leave blank to end): PUSHOVER_TOKEN       
Enter secret value for PUSHOVER_TOKEN: a      
Enter secret name (leave blank to end):      
Create requirements.txt file? (y/n) [n]: n      
Create Github Action to automatically update Space on 'git push'? [n]: n      
Space available at https://huggingface.co/spaces/ranjitsolo/career_conversation      
     
Got an Error     
ModuleNotFoundError: No module named 'dotenv'      